In [ ]:
import optuna
import matplotlib.pylab as plt
import seaborn as sns
import pandas as pd

In [ ]:
dfs = []
study_names = ['ice_fishing_model_11_25_cma-es_3.0_0.1',
               'ice_fishing_model_11_25_cma-es_2.5_0.1',
               'ice_fishing_model_11_25_cma-es_2.0_0.1']

for s in study_names:
    storage_name = 'sqlite:///ice_fishing_model_11_25.db'

    study = optuna.create_study(
        study_name=s, 
        storage=storage_name,
        sampler=optuna.samplers.CmaEsSampler(n_startup_trials=500),
        load_if_exists=True)
    dfs.append(study.trials_dataframe(attrs=("number", "value", "params", "state", 'user_attrs')))

In [ ]:
dfs[0].head()

In [ ]:
dfs = [df[(df['state'] == 'COMPLETE') & (df['number'] >= 500)] for df in dfs]

In [ ]:
dfs[0].value.hist()

In [ ]:
top_perc = 0.2

top_20_dfs = [df.sort_values(by='value', ascending=False).head(int(len(df) * top_perc)).reset_index() for df in dfs]

In [ ]:
col_orig = ['value', 'user_attrs_fish_abundance', 'user_attrs_ss_tau',  
            'params_slw_base', 'params_slw_fish', 'params_slw_soc', 'params_slw_time',
            'user_attrs_ssw_fail', 'user_attrs_ssw_loc', 'user_attrs_ssw_soc', 'user_attrs_ssw_suc',
            'params_ssw_fail_ls', 'params_ssw_suc_ls', 'params_ssw_soc_ls']

params = ['slw_base', 'slw_fish', 'slw_soc', 'slw_time',
          'ssw_fail', 'ssw_loc', 'ssw_soc', 'ssw_suc', 
          'ssw_fail_ls', 'ssw_suc_ls', 'ssw_soc_ls']
col_new = ['catch', 'fish_abundance', 'ss_tau'] + params

col_name_mapping = {i:j for i, j in zip(col_orig, col_new)}

In [ ]:
top_20_percent = [d[col_orig].rename(columns=col_name_mapping) for d in top_20_dfs]
top_20_percent = [d.replace({'fish_abundance': {3.0: 'low', 2.5: 'middle', 2.0: 'high'}}) for d in top_20_percent]

In [ ]:
top_20_percent[0].head()

In [ ]:
_top_20_percent = top_20_percent.copy()

In [ ]:
# select fail for middle abundance < 0.35
# _top_20_percent[1] = _top_20_percent[1][_top_20_percent[1]['ssw_fail'] > 0.35]
_top_20_percent = pd.concat(_top_20_percent)

In [ ]:
top_20_percent_melted = _top_20_percent.loc[:, params + ['fish_abundance']].melt(
     id_vars=['fish_abundance'], var_name='parameter', value_name='value')

In [ ]:
top_20_percent_melted.head()

In [ ]:
g = sns.catplot(
    data=top_20_percent_melted,
    x='fish_abundance',
    y='value',
    hue='fish_abundance',
    col='parameter',
    kind='point',  # 'point',
    errorbar='sd',
    # errorbar='ci',
    sharey=False,
    col_wrap=4,
    height=3,
    aspect=0.7,
    legend=True
)

# Set titles to just the parameter name for easy matching
g.set_titles("{col_name}")

# Define manual limits
limits = {
    'slw_base': (-20, -1), 
    'slw_fish': (-15.0, 0.0), 
    'slw_soc': (-3, 3.0), 
    'slw_time': (0.0, 2.0),
    'ssw_fail': (0, 0.6),
    'ssw_loc':  (0, 0.6),
    'ssw_soc':  (0, 0.6),
    'ssw_suc':  (0, 0.6),
    'ssw_fail_ls': (0.5, 50), 
    'ssw_soc_ls': (0.5, 50),
    'ssw_suc_ls': (0.5, 50)
}

ref_lines = {
    'slw_base': -3,
    'slw_fish': -1.72,
    'slw_soc': 0,  # -0.33,
    'slw_time': 0.13,
    # 'ssw_fail': 0.25,
    # 'ssw_loc': 0.25,
    # 'ssw_soc': 0.25,
    # 'ssw_suc': 0.25,
    'ssw_fail_ls': 10,
    'ssw_suc_ls': 10,
    'ssw_soc_ls': 25
}

# Apply limits
for ax, col_name in zip(g.axes.flat, g.col_names):
    
    # 1. Apply Limits
    if col_name in limits:
        ax.set_ylim(limits[col_name])

    # 2. Draw Horizontal Lines
    if col_name in ref_lines:
        ax.axhline(
            y=ref_lines[col_name], 
            color='red', 
            linestyle='--', 
            linewidth=2, 
            alpha=0.8,
            label='Target' # Optional: adds this line to a potential legend
        )
        
sns.move_legend(
    g, 
    "lower right", 
    bbox_to_anchor=(0.93, 0.2), # Adjust the 0.02 slightly if it's cut off
    frameon=True,             # Draws a box around the legend
    title="Fish abundance"            # Ensure title is correct
)
        
plt.tight_layout()
plt.savefig('optimization_results.png', dpi=300)
plt.show()

In [ ]:
middle_df = top_20_percent[1].copy()

In [ ]:
middle_df['strategy'] = middle_df['ssw_fail'] < 0.35
middle_df.replace({'strategy': {True: '1', False: '2'}}, inplace=True)

In [ ]:
middle_df_melted = middle_df.loc[:, params + ['strategy']].melt(
     id_vars=['strategy'], var_name='parameter', value_name='value')

In [ ]:
g = sns.catplot(
    data=middle_df_melted,
    x='strategy',
    y='value',
    hue='strategy',
    col='parameter',
    kind='point',  # 'point',
    errorbar='sd',
    # errorbar='ci',
    sharey=False,
    col_wrap=4,
    height=3,
    aspect=0.7,
    legend=True
)

# Set titles to just the parameter name for easy matching
g.set_titles("{col_name}")

# Define manual limits
limits = {
    'slw_base': (-20, -1), 
    'slw_fish': (-15.0, 0.0), 
    'slw_soc': (-3, 3.0), 
    'slw_time': (0.0, 2.0),
    'ssw_fail': (0, 0.6),
    'ssw_loc':  (0, 0.6),
    'ssw_soc':  (0, 0.6),
    'ssw_suc':  (0, 0.6),
    'ssw_fail_ls': (0.5, 50), 
    'ssw_soc_ls': (0.5, 50),
    'ssw_suc_ls': (0.5, 50)
}

ref_lines = {
    'slw_base'   : middle_df.loc[middle_df['strategy'] == '1', 'slw_base'   ].mean(),
    'slw_fish'   : middle_df.loc[middle_df['strategy'] == '1', 'slw_fish'   ].mean(),
    'slw_soc'    : middle_df.loc[middle_df['strategy'] == '1', 'slw_soc'    ].mean(),
    'slw_time'   : middle_df.loc[middle_df['strategy'] == '1', 'slw_time'   ].mean(),
    'ssw_fail'   : middle_df.loc[middle_df['strategy'] == '1', 'ssw_fail'   ].mean(),
    'ssw_loc'    : middle_df.loc[middle_df['strategy'] == '1', 'ssw_loc'    ].mean(),
    'ssw_soc'    : middle_df.loc[middle_df['strategy'] == '1', 'ssw_soc'    ].mean(),
    'ssw_suc'    : middle_df.loc[middle_df['strategy'] == '1', 'ssw_suc'    ].mean(),
    'ssw_fail_ls': middle_df.loc[middle_df['strategy'] == '1', 'ssw_fail_ls'].mean(),
    'ssw_suc_ls' : middle_df.loc[middle_df['strategy'] == '1', 'ssw_suc_ls' ].mean(),
    'ssw_soc_ls' : middle_df.loc[middle_df['strategy'] == '1', 'ssw_soc_ls' ].mean(),
}

# Apply limits
for ax, col_name in zip(g.axes.flat, g.col_names):
    
    # 1. Apply Limits
    if col_name in limits:
        ax.set_ylim(limits[col_name])

    # 2. Draw Horizontal Lines
    if col_name in ref_lines:
        ax.axhline(
            y=ref_lines[col_name], 
            color='gray', 
            linestyle='--', 
            linewidth=2, 
            alpha=0.3,
            label='Target' # Optional: adds this line to a potential legend
        )

sns.move_legend(
    g, 
    "lower right", 
    bbox_to_anchor=(0.93, 0.2), # Adjust the 0.02 slightly if it's cut off
    frameon=True,             # Draws a box around the legend
    # title="Fish abundance"            # Ensure title is correct
)
        
plt.tight_layout()
plt.savefig('optimization_results_middle.png', dpi=300)
plt.show()

In [ ]:
_top_20_percent.head()

In [ ]:
g = sns.catplot(
    data=_top_20_percent,
    x='fish_abundance',
    y='catch',
    hue='fish_abundance',
    col='fish_abundance',
    kind='point',  # 'point',
    errorbar='sd',
    # errorbar='ci',
    sharey=False,
    # sharex=False,
    col_wrap=4,
    height=3,
    aspect=0.7,
    legend=True
)

# Set titles to just the parameter name for easy matching
g.set_titles("{col_name}")

ref_lines = {
    'low': 25,
    'middle': 60,
    'high': 120,
}

# Apply limits
for ax, col_name in zip(g.axes.flat, g.col_names):
    
    # 1. Apply Limits
    if col_name in limits:
        ax.set_ylim(limits[col_name])

    # 2. Draw Horizontal Lines
    if col_name in ref_lines:
        ax.axhline(
            y=ref_lines[col_name], 
            color='red', 
            linestyle='--', 
            linewidth=2, 
            alpha=0.8,
        )
        
plt.tight_layout()
plt.savefig('optimization_results_performance.png', dpi=300)
plt.show()